In [1]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib
import data
import feature_engineering

warnings.simplefilter(action='ignore', category=FutureWarning)

# Read Prices

In [2]:
df_equity = data.read_excel_sheets()
df_equity['OpenTime'] = pd.to_datetime(df_equity['OpenTime'])
df_equity = df_equity.set_index('OpenTime') 

list_equity = df_equity.Symbol.unique().tolist()
print(f"\nEquity:\n{list_equity}")

crypto = 'data_crypto.xlsx'
df_crypto = (
    pd.read_excel(crypto)
    .drop(columns=['Unnamed: 0'])
)

df_crypto['OpenTime'] = pd.to_datetime(df_crypto['OpenTime'])
df_crypto = df_crypto.set_index('OpenTime') 

list_crypto = df_crypto.Symbol.unique().tolist()
print(f"\nCrypto:\n{list_crypto}")

AAPL
MSFT
NVDA
AMZN
META
GOOGL
BRK.B
GOOG
AVGO
TSLA
SPY_ETF
DAXEX_ETF
CAC40_ETF
NKY225_ETF
DJIA_ETF
EQQQ_ETF
FTSE_ETF

Equity:
['NKY225_ETF', 'AAPL', 'AMZN', 'AVGO', 'BRK.B', 'DJIA_ETF', 'GOOG', 'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY_ETF', 'TSLA', 'DAXEX_ETF', 'CAC40_ETF', 'EQQQ_ETF', 'FTSE_ETF']

Crypto:
['ADAUSDT', 'AVAXUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'SOLUSDT', 'TRXUSDT', 'USDCUSDT', 'XRPUSDT']


**Sample Pairs**

```
Pairs: META & BNBUSDT
Correlation: 0.787

Spread ADF Statistic: -3.3381
p-value: 0.013 (Spread is stationary)

Cointegration Test Statistic: -3.6076
p-value: 0.024 (Both series are cointegrated)

In [3]:
ticker_eqt = 'META'
ticker_cpy = 'BNBUSDT'

close = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Close')
close

,META,BNBUSDT
OpenTime,,
2024-10-14 12:30:00,593.1400,590.50
2024-10-14 12:35:00,593.5200,590.10
2024-10-14 12:40:00,594.7200,589.30
2024-10-14 12:45:00,595.3300,588.70
2024-10-14 12:50:00,596.4400,588.30
...,...,...
2025-01-17 18:35:00,614.0775,724.02
2025-01-17 18:40:00,613.7600,724.30
2025-01-17 18:45:00,614.0500,723.72


In [6]:
config = {
    'ema': [12, 26],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14],
    'bb': {
        'timeperiod': 20,
        'nbdevup': 2,
        'nbdevdn': 2,
    },
    'atr': {
        'timeperiod': 14,
    },
    'stoch': {
        'fastk_period': 14,
        'slowk_period': 3,
        'slowd_period': 3,
    },
    'cci': {
        'timeperiod': 20,
    },
    'willr': {
        'timeperiod': 14,
    },
}

ticker_eqt = 'META'
ticker_cpy = 'BNBUSDT'


feat = feature_engineering.create_features(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), config)

In [7]:
feat

,META_EMA12,META_EMA26,META_MACD,META_RSI14,META_BBupper,META_BBmiddle,META_BBlower,META_ATR14,META_StochK14,META_StochD3,...,BNBUSDT_RSI14,BNBUSDT_BBupper,BNBUSDT_BBmiddle,BNBUSDT_BBlower,BNBUSDT_ATR14,BNBUSDT_StochK14,BNBUSDT_StochD3,BNBUSDT_CCI20,BNBUSDT_WILLR14,NormalizedSpread
OpenTime,,,,,,,,,,,,,,,,,,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.605293
2024-10-14 12:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625364
2024-10-14 12:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.676829
2024-10-14 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.707964
2024-10-14 12:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.746820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,-0.056616,50.800371,614.888490,613.825295,612.762100,0.431433,86.574607,87.758282,...,53.412849,725.503379,723.7675,722.031621,0.631671,48.044976,38.085423,23.742360,-36.466165,-1.291689
2025-01-17 18:40:00,613.903332,613.969266,-0.065935,48.078678,614.631796,613.756295,612.880794,0.423295,82.951882,86.521331,...,54.948980,725.398232,723.7335,722.068768,0.606552,70.631497,52.400354,56.520004,-0.000000,-1.307064
2025-01-17 18:45:00,613.925896,613.975247,-0.049351,50.677954,614.573134,613.738290,612.903446,0.413774,82.259302,83.928597,...,51.184095,725.302839,723.6885,722.074161,0.604655,78.030737,65.569070,3.419082,-29.441624,-1.284677
